# PIVOT und UNPIVOT

PIVOT Tabellen fassen Daten übersichtlich zusammen. Sie haben das Werkzeug wahrscheinlich schon einmal in Excel kennengelernt.

Die Tabelle PivotDaten in der SQL\_Seminar Datenbank zeigt Ihnen das Ergebnis einer Pivot-Anweisung in T-SQL.

In [ ]:
-- Rohdaten für PIVOT Demo in Excel
SELECT k.Kategorie
, b.BestellungID
, b.Bestelldatum 
, CAST(d.Verkaufspreis * d.Menge AS money) AS Umsatz 
FROM Produktkategorie AS k
JOIN Produkt AS p ON k.KategorieID = p.KategorieID
JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
JOIN Bestellung AS b ON d.BestellungID = b.BestellungID;

In [ ]:
-- Rohdaten für PIVOT Abfrage
-- Die einzelnen Umsätze werden ohne Aggregation ausgegeben (KEINE Summe!)
SELECT k.Kategorie
    , YEAR(b.Bestelldatum) AS Jahr
    , CAST(d.Verkaufspreis * d.Menge AS money) AS Umsatz 
FROM Produktkategorie AS k
JOIN Produkt AS p ON k.KategorieID = p.KategorieID
JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
JOIN Bestellung AS b ON d.BestellungID = b.BestellungID

In [ ]:
-- Pivot mit Umsatzdaten in einer vorangestellten Unterabfrage
WITH Umsatzdaten AS 
(
  SELECT k.Kategorie
    , YEAR(b.Bestelldatum) AS Jahr
    , CAST(d.Verkaufspreis * d.Menge AS money) AS Umsatz 
  FROM Produktkategorie AS k
  JOIN Produkt AS p ON k.KategorieID = p.KategorieID
  JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
  JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
)
SELECT [Kategorie], [2016], [2017], [2018]
FROM Umsatzdaten
PIVOT 
(
	SUM(Umsatz) FOR Jahr IN ([2016], [2017], [2018])
) AS pt
ORDER BY Kategorie;

In [ ]:
-- Pivot mit Umsatzdaten in einer Unterabfrage in der FROM-Klausel
SELECT [Kategorie], [2016], [2017], [2018]
FROM (
  SELECT k.Kategorie
    , YEAR(b.Bestelldatum) AS Jahr
    , CAST(d.Verkaufspreis * d.Menge AS money) AS Umsatz 
  FROM Produktkategorie AS k
  JOIN Produkt AS p ON k.KategorieID = p.KategorieID
  JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
  JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
) AS Umsatzdaten
PIVOT 
(
	SUM(Umsatz) FOR Jahr IN ([2016], [2017], [2018])
) AS pt -- Alias
ORDER BY Kategorie;

In [ ]:
-- Ersetzen von NULL in der Ausgabe
WITH Umsatzdaten AS 
(
  SELECT k.Kategorie
    , YEAR(b.Bestelldatum) AS Jahr
    , CAST(d.Verkaufspreis * d.Menge AS money) AS Umsatz 
  FROM Produktkategorie AS k
  JOIN Produkt AS p ON k.KategorieID = p.KategorieID
  JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
  JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
)
SELECT [Kategorie]
  , COALESCE([2016], 0) AS [2016]
  , COALESCE([2017], 0) AS [2017]
  , COALESCE([2018], 0) AS [2018]
  , COALESCE([2019], 0) AS [2019]
  , COALESCE([2020], 0) AS [2020]
FROM Umsatzdaten
PIVOT 
(
	SUM(Umsatz) FOR Jahr IN ([2016], [2017], [2018], [2019], [2020])
) AS pt -- Alias
ORDER BY Kategorie;

In [ ]:
-- Ansicht der gleichen Daten mit vertauschten Spalten und Zeilen
WITH Umsatzdaten AS (
  SELECT k.Kategorie
    , YEAR(b.Bestelldatum) AS Jahr
    , CAST(d.Verkaufspreis * d.Menge AS money) AS Umsatz 
  FROM Produktkategorie AS k
  JOIN Produkt AS p ON k.KategorieID = p.KategorieID
  JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
  JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
)
SELECT [Jahr], [Fleischprodukte],[Getränke],[Getreideprodukte],[Gewürze],[Meeresfrüchte],[Milchprodukte],[Naturprodukte],[Süßwaren]
FROM Umsatzdaten
PIVOT (SUM(Umsatz) FOR Kategorie IN([Fleischprodukte],[Getränke],[Getreideprodukte],[Gewürze],[Meeresfrüchte],[Milchprodukte],[Naturprodukte],[Süßwaren])) AS pt
ORDER BY Jahr;

In [ ]:
-- UNPIVOT Vorbereiten
DROP TABLE IF EXISTS PivotDaten;
GO

WITH Umsatzdaten AS (
  SELECT k.Kategorie
    , YEAR(b.Bestelldatum) AS Jahr
    , CAST(d.Verkaufspreis * d.Menge AS money) AS Umsatz 
  FROM Produktkategorie AS k
  JOIN Produkt AS p ON k.KategorieID = p.KategorieID
  JOIN BestellDetail AS d ON p.ProduktID = d.ProduktID
  JOIN Bestellung AS b ON d.BestellungID = b.BestellungID
)
SELECT [Kategorie], [2016], [2017], [2018]
INTO PivotDaten
FROM Umsatzdaten
PIVOT (SUM(Umsatz) FOR Jahr IN([2016], [2017], [2018])) AS pt
ORDER BY Kategorie;

SELECT * FROM PivotDaten;

In [ ]:
-- UNPIVOT
SELECT Kategorie, Jahr, Umsatz
FROM ( 
  SELECT Kategorie, [2016], [2017], [2018] FROM PivotDaten
  ) AS pvt
UNPIVOT (
  Umsatz FOR Jahr IN ([2016], [2017], [2018])
) AS unpvt;